In [2]:
import pandas as pd

average_interest_rates_df = pd.read_csv("CNB_prumer_dim_len_roky.csv", encoding="utf-8")
salaries_and_inflation_df = pd.read_csv("Inflace_a_platy_FACT.csv", encoding="utf-8")
property_df = pd.read_csv("Brno_vycistene_udaje.csv", encoding="utf-8")

property_df = property_df[property_df["typ_nabidky"].str.lower() == "sale"]

# Vytvorenie stĺpcov "cena_90" a "nasetrene_10"
property_df["cena_90"] = property_df["cena"] * 0.90
property_df["nasetrene_10"] = property_df["cena"] * 0.10

# Spojenie tabuliek na základe stĺpca "rok"
merged_df = pd.merge(property_df, average_interest_rates_df[["rok", "rocni_prumer"]], on="rok", how="left")
merged_df = pd.merge(merged_df, salaries_and_inflation_df[["rok", "platy_muzi", "platy_zeny"]], on="rok", how="left")


merged_df["platy_muzi"] = pd.to_numeric(merged_df["platy_muzi"].str.replace(" ", "").str.replace(",", "."), errors="coerce")
merged_df["platy_zeny"] = pd.to_numeric(merged_df["platy_zeny"].str.replace(" ", "").str.replace(",", "."), errors="coerce")

# Predpokladané ročné úrokové sadzby a dĺžky splácania hypoték
loan_term_years = 30  # 30 rokov

# Výpočet mesačnej splátky hypotéky
def calculate_monthly_payment(principal, interest_rate, term_years):
    monthly_rate = interest_rate / 12
    num_payments = term_years * 12
    monthly_payment = principal * (monthly_rate * (1 + monthly_rate) ** num_payments) / ((1 + monthly_rate) ** num_payments - 1)
    return monthly_payment

# Výpočet mesačných splátok hypotéky s úrokovými sadzbami za jednotlivé roky
merged_df["mesicni_splatka_90"] = merged_df.apply(
    lambda row: calculate_monthly_payment(row["cena_90"], row["rocni_prumer"] / 100, loan_term_years),
    axis=1
)
# Predpoklad maximálnej mesačnej splátky hypotéky (40% platu)
merged_df["max_splatka_muzi"] = merged_df["platy_muzi"] * 0.40
merged_df["max_splatka_zeny"] = merged_df["platy_zeny"] * 0.40
merged_df["max_splatka_par"] = (merged_df["platy_muzi"] + merged_df["platy_zeny"]) * 0.40

# Výpočet pomeru mesačných splátok k platu
merged_df["splatka_k_platu_muzi"] = merged_df["mesicni_splatka_90"] / merged_df["max_splatka_muzi"]
merged_df["splatka_k_platu_zeny"] = merged_df["mesicni_splatka_90"] / merged_df["max_splatka_zeny"]
merged_df["splatka_k_platu_par"] = merged_df["mesicni_splatka_90"] / merged_df["max_splatka_par"]

# Pridanie stĺpcov s hodnotami TRUE/FALSE
merged_df["muzi_si_mohou_dovolit"] = merged_df["splatka_k_platu_muzi"] <= 1
merged_df["zeny_si_mohou_dovolit"] = merged_df["splatka_k_platu_zeny"] <= 1
merged_df["par_si_muze_dovolit"] = merged_df["splatka_k_platu_par"] <= 1

# Výpočet priemerných platov a maximálnych splátok pre každý rok, vrátane dostupnosti splácania
years = merged_df["rok"].unique()
results = []

for year in years:
    yearly_data = merged_df[merged_df["rok"] == year]
    average_salary_male = yearly_data["platy_muzi"].mean()
    average_salary_female = yearly_data["platy_zeny"].mean()
    max_payment_male = average_salary_male * 0.40
    max_payment_female = average_salary_female * 0.40
    max_payment_couple = (average_salary_male + average_salary_female) * 0.40

    can_afford_male = yearly_data["muzi_si_mohou_dovolit"].all()
    can_afford_female = yearly_data["zeny_si_mohou_dovolit"].all()
    can_afford_couple = yearly_data["par_si_muze_dovolit"].all()
    
    results.append({
        "rok": year,
        "prumerny_plat_muzi": average_salary_male,
        "prumerny_plat_zeny": average_salary_female,
        "max_splatka_muzi": max_payment_male,
        "max_splatka_zeny": max_payment_female,
        "max_splatka_par": max_payment_couple,
        "muzi_si_mohou_dovolit": can_afford_male,
        "zeny_si_mohou_dovolit": can_afford_female,
        "par_si_muze_dovolit": can_afford_couple
    })

results_df = pd.DataFrame(results)
results_df = merged_df[["rok", "cena", "mena","rocni_prumer", "forma_platby", "nasetrene_10", "cena_90", 
                        "mesicni_splatka_90", "max_splatka_muzi", "max_splatka_zeny", "max_splatka_par", "muzi_si_mohou_dovolit", 
                        "zeny_si_mohou_dovolit", "par_si_muze_dovolit", "dipozice", "uzitna_plocha", "typ_nemovitosti"]]
results_df.to_csv("Hypoteka_FACT.csv", index=False, encoding="utf-8")